In [ ]:
"""
Program expects Data/A/100/something.pdf structure
Create a "splitted" folder for the output and then run the program
Splits the pages in half if the longest line is less than 60 characters a.k.a
there are 2 columns of text
"""
import pdfminer
import pdfrw
import alive_progress

import sys
import os
from pdfminer.high_level import extract_text
 
directory = "./data/cla_pdfs"
from pdfrw import PdfReader, PdfWriter, PageMerge

#directory to put the pdfs that have been split
split_dir= './data/split_pdfs/'
os.mkdir(split_dir)

#function to split pdf pages in half
def splitpage(src):
    """Split a page into two (left and right)"""
    # Yield a result for each half of the page
    for x_pos in (0, 0.5):
        yield PageMerge().add(src, viewrect=(x_pos, 0, 0.5, 1)).render()
 
#count of split pdfs and dictionary with totals after each folder is processed
split = 0
split_dict = {}

#get list of folders from relevant directory that has pdfs needing to be split 
jc_numbers= os.listdir(directory)
print(jc_numbers)
for dir in jc_numbers:
    print(dir)

    for document in os.listdir(f"{directory}/{dir}"):
        try:
            text = extract_text(f"{directory}/{dir}/{document}")
            text = text.splitlines()
            if len(max(text, key=len)) < 60:
                writer = PdfWriter()
                page_number = 0
                for page in PdfReader(f"{directory}/{dir}/{document}").pages:
                    writer.addpages(splitpage(page))
                
                if os.path.exists(f"{split_dir}/{dir}"):
                    writer.write(f"{split_dir}/{dir}/{document}")
                    split += 1
                else:
                    os.mkdir(f"{split_dir}/{dir}")
                    writer.write(f"{split_dir}/{dir}/{document}")
                    split += 1
        except:
            print('error at directory: ' + str(dir))
            pass
    split_dict[dir] = split           
    print(f"Split pdfs: {split}")
    print(split_dict)

In [3]:
#if running functions after eachother this path is not needed, can be split_dir
#but otherwise make this path 
path = "/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_pdfs"

#join pdf half pages into full pdfs, in this case by language
pc_list = os.listdir(path)
for pc_folder in os.listdir(path):
    for document in os.listdir(f"{path}/{pc_folder}"):
        writer_NL = PdfWriter()
        writer_FR = PdfWriter()
        page_number = 0
        for page in PdfReader(f"{path}/{pc_folder}/{document}").pages:
            page_number += 1
            if page_number%2 == 1:
                writer_NL.addpage(page)
            else:
                writer_FR.addpage(page)
        try:
            os.mkdir('./data/split_nl/' + str(document)[0:3])
            os.mkdir('./data/split_fr/' + str(document)[0:3])
            writer_NL.write(f"./data/split_nl/" + str(document)[0:3] + f"/NL_{document}")
            writer_FR.write(f"./data/split_fr/" + str(document)[0:3]+ f"/FR_{document}")
        except:
            writer_NL.write(f"./data/split_nl/" + str(document)[0:3] + f"/NL_{document}")
            writer_FR.write(f"./data/split_fr/" + str(document)[0:3]+ f"/FR_{document}")
        else:
            print(('document' + str(document) + 'not made'))
            pass         

FileNotFoundError: [Errno 2] No such file or directory: 'data/split_pdfs'